In [55]:
tf.reset_default_graph()

In [1]:
import tensorflow as tf
import numpy as np
from keras.preprocessing.image import ImageDataGenerator,load_img,img_to_array
from sklearn.datasets import load_files
from keras.utils import to_categorical
import cv2

C:\Users\Gowtham\Anaconda3\envs\dog-project\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
train_path = 'data/train'
valid_path = 'data/valid'
test_path = 'data/test'

In [3]:
# loading images for paths
def load_images(path):
    i = load_files(path)
    return i['filenames'],i['target']

In [4]:
train_images,train_targets = load_images(train_path)
valid_images,valid_targets = load_images(valid_path)
test_images,test_targets = load_images(test_path)

In [5]:
# converting raw images to tensors
def split(raw_image):
    img = tf.contrib.keras.preprocessing.image.load_img(raw_image)
    img = tf.contrib.keras.preprocessing.image.img_to_array(img)
    img = cv2.resize(img,(256,256))/255.
    return img
def image_to_tensors(raw_images):
    a = list()
    for i in range(len(raw_images)):
        img = split(raw_images[i])
        a.append(img)
    a = np.array(a)
    return a

In [6]:
train_tensors = image_to_tensors(train_images)
valid_tensors = image_to_tensors(valid_images)
test_tensors = image_to_tensors(test_images)

In [7]:
train_datagen = ImageDataGenerator(rotation_range=60,width_shift_range=0.3,height_shift_range=0.3,
                             rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True,fill_mode='Nearest')

In [8]:
train_datagen.fit(train_tensors)
#x = train_datagen.flow(train_tensors,train_targets,batch_size=50)
for e in range(1):
    batch = 0
    for x_batch,y_batch in train_datagen.flow(train_tensors,train_targets,batch_size=50):
        batch += 1
        print(x_batch.shape)
        print(y_batch.shape)
        if batch > train_tensors.shape[0]//50:
            break

RuntimeError: boundary mode not supported

In [16]:
def one_hot(targets):
    targets = to_categorical(targets)
    return targets

In [17]:
train_targets = one_hot(train_targets)
valid_targets = one_hot(valid_targets)
test_targets = one_hot(test_targets)

In [18]:
train_targets[1]

array([0., 1., 0.])

In [19]:
def generate_filters(kernel_size,in_depth,out_depth):
    return tf.Variable(tf.random_uniform([kernel_size,kernel_size,in_depth,out_depth],minval=0.25,maxval=0.75))
def generate_biases(out_depth):
    return tf.Variable(tf.random_uniform([out_depth],minval=0.25,maxval=0.75))
def generate_weights_for_fc(in_nodes,out_nodes):
    return tf.Variable(tf.random_uniform([in_nodes,out_nodes],minval=0.25,maxval=0.75))
'''
weights = {
    'wc1': tf.Variable(tf.random_uniform([3,3,3,16],minval=0.25,maxval=0.75)),
    'wc2': tf.Variable(tf.random_uniform([3,3,16,32],minval=0.25,maxval=0.75)),
    'wc3': tf.Variable(tf.random_uniform([3,3,32,64],minval=0.25,maxval=0.75)),
    'wc4': tf.Variable(tf.random_uniform([3,3,64,128],minval=0.25,maxval=0.75)),
    'wc5': tf.Variable(tf.random_uniform([3,3,128,256],minval=0.25,maxval=0.75)),
    'fc1': tf.Variable(tf.random_uniform([16*16*256,1024],minval=0.25,maxval=0.75)),
    'fc2': tf.Variable(tf.random_uniform([1024,3],minval=0.25,maxval=0.75))
}
biases = {
    'bc1': tf.Variable(tf.random_uniform([16],minval=0.25,maxval=0.75)),
    'bc2': tf.Variable(tf.random_uniform([32],minval=0.25,maxval=0.75)),
    'bc3': tf.Variable(tf.random_uniform([64],minval=0.25,maxval=0.75)),
    'bc4': tf.Variable(tf.random_uniform([128],minval=0.25,maxval=0.75)),
    'bc5': tf.Variable(tf.random_uniform([256],minval=0.25,maxval=0.75)),
    'bf1': tf.Variable(tf.random_uniform([1024],minval=0.25,maxval=0.75)),
    'bf2': tf.Variable(tf.random_uniform([3],minval=0.25,maxval=0.75))
}
'''

"\nweights = {\n    'wc1': tf.Variable(tf.random_uniform([3,3,3,16],minval=0.25,maxval=0.75)),\n    'wc2': tf.Variable(tf.random_uniform([3,3,16,32],minval=0.25,maxval=0.75)),\n    'wc3': tf.Variable(tf.random_uniform([3,3,32,64],minval=0.25,maxval=0.75)),\n    'wc4': tf.Variable(tf.random_uniform([3,3,64,128],minval=0.25,maxval=0.75)),\n    'wc5': tf.Variable(tf.random_uniform([3,3,128,256],minval=0.25,maxval=0.75)),\n    'fc1': tf.Variable(tf.random_uniform([16*16*256,1024],minval=0.25,maxval=0.75)),\n    'fc2': tf.Variable(tf.random_uniform([1024,3],minval=0.25,maxval=0.75))\n}\nbiases = {\n    'bc1': tf.Variable(tf.random_uniform([16],minval=0.25,maxval=0.75)),\n    'bc2': tf.Variable(tf.random_uniform([32],minval=0.25,maxval=0.75)),\n    'bc3': tf.Variable(tf.random_uniform([64],minval=0.25,maxval=0.75)),\n    'bc4': tf.Variable(tf.random_uniform([128],minval=0.25,maxval=0.75)),\n    'bc5': tf.Variable(tf.random_uniform([256],minval=0.25,maxval=0.75)),\n    'bf1': tf.Variable(tf.r

In [20]:
learning_rate = 0.00001
epochs = 50
batch_size = 100
dropout = 0.75

In [27]:
def create_conv2d(image_batch,in_depth,out_depth):
    kernel_size = 3
    layer = tf.nn.conv2d(input=image_batch,filter=generate_filters(kernel_size,in_depth,out_depth),strides=[1,1,1,1],padding='SAME')
    layer = tf.nn.bias_add(value=layer,bias=generate_biases(out_depth))
    layer = tf.nn.relu(layer)
    return layer
def create_maxpool(conv_layer):
    layer = tf.nn.max_pool(conv_layer,[1,3,3,1],[1,2,2,1],padding='SAME')
    return layer
def create_fc_after_conv(conv_layer,in_nodes,out_nodes,keep_prob):
    layer = tf.reshape(conv_layer,[-1,16*16*256])
    layer = tf.add(tf.matmul(layer,generate_weights_for_fc(in_nodes,out_nodes)),generate_biases(out_nodes))
    layer = tf.nn.relu(layer)
    layer = tf.nn.dropout(layer,keep_prob)
    return layer
def create_fc_after_fc(fc,in_nodes,out_nodes,keep_prob):
    layer = tf.add(tf.matmul(fc,generate_weights_for_fc(in_nodes,out_nodes)),generate_biases(out_nodes))
    layer = tf.nn.relu(layer)
    layer = tf.nn.dropout(layer,keep_prob)
    return layer
def next_batch(image_batch,batch,batch_size):
    start = batch*batch_size
    end = start+batch_size
    return image_batch[start:end]

In [28]:
def convnet(x,keep_prob):
    conv1 = create_conv2d(x,3,16)
    max1 = create_maxpool(conv1)
    conv2 = create_conv2d(max1,16,32)
    max2 = create_maxpool(conv2)
    conv3 = create_conv2d(max2,32,64)
    max3 = create_maxpool(conv3)
    conv4 = create_conv2d(max3,64,128)
    max4 = create_maxpool(conv4)
    conv5 = create_conv2d(max4,128,256)
    fc1 = create_fc_after_conv(conv5,16*16*256,1024,dropout)
    fc2 = create_fc_after_fc(fc1,1024,2048,dropout)
    fc3 = tf.add(tf.matmul(fc2,generate_weights_for_fc(2048,3)),generate_biases(3))
    return fc3

In [29]:
x = tf.placeholder(tf.float32,[None,256,256,3])
y = tf.placeholder(tf.float32,[None,3])
keep_prob = tf.placeholder(tf.float32)

logits = convnet(x,keep_prob)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits,labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

correct_pred = tf.equal(tf.argmax(logits,1),tf.argmax(y,1))

accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32))

init = tf.global_variables_initializer()
config = tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.allocator_type ='BFC'
config.gpu_options.allow_growth = True
with tf.Session(config=config) as sess:
    sess.run(init)
    for epoch in range(epochs):
        for batch in range(train_tensors.shape[0]//batch_size):
            batch_x = next_batch(train_tensors,batch,batch_size)
            batch_y = next_batch(train_targets,batch,batch_size)
            
            sess.run(optimizer,feed_dict={
                x: batch_x,
                y: batch_y,
                keep_prob:dropout
            })
            loss = sess.run(cost,feed_dict={
                x: batch_x,
                y: batch_y,
                keep_prob: 1.
            })
            valid_acc = sess.run(accuracy,feed_dict={
                x:valid_tensors[:],
                y:valid_targets[:],
                keep_prob:1.
            })
            
            print('Epoch {:>2}, Batch{:>3},loss {:>3},Validation_accuracy{:3}'.format(epoch+1,batch+1,loss,valid_acc))
            

Epoch  1, Batch  1,loss 4.730435549896088e+25,Validation_accuracy0.20000000298023224
Epoch  1, Batch  2,loss 191920127082496.0,Validation_accuracy0.20000000298023224
Epoch  1, Batch  3,loss 4.37773174529065e+16,Validation_accuracy0.5066666603088379
Epoch  1, Batch  4,loss 1.2089728116989136,Validation_accuracy0.2800000011920929
Epoch  1, Batch  5,loss 1.20652437210083,Validation_accuracy0.2800000011920929
Epoch  1, Batch  6,loss 1.1952263116836548,Validation_accuracy0.2800000011920929
Epoch  1, Batch  7,loss 1.1763986349105835,Validation_accuracy0.2800000011920929
Epoch  1, Batch  8,loss 1.2017619609832764,Validation_accuracy0.2800000011920929
Epoch  1, Batch  9,loss 1.1605372428894043,Validation_accuracy0.2800000011920929
Epoch  1, Batch 10,loss 1.2068405151367188,Validation_accuracy0.2800000011920929
Epoch  1, Batch 11,loss 1.190789818763733,Validation_accuracy0.2800000011920929
Epoch  1, Batch 12,loss 1.2041985988616943,Validation_accuracy0.2800000011920929
Epoch  1, Batch 13,loss 1

Epoch  6, Batch  3,loss 1.2037935256958008,Validation_accuracy0.2800000011920929
Epoch  6, Batch  4,loss 1.2087081670761108,Validation_accuracy0.2800000011920929
Epoch  6, Batch  5,loss 1.2062463760375977,Validation_accuracy0.2800000011920929
Epoch  6, Batch  6,loss 1.1949690580368042,Validation_accuracy0.2800000011920929
Epoch  6, Batch  7,loss 1.1761759519577026,Validation_accuracy0.2800000011920929
Epoch  6, Batch  8,loss 1.2015063762664795,Validation_accuracy0.2800000011920929
Epoch  6, Batch  9,loss 1.160302996635437,Validation_accuracy0.2800000011920929
Epoch  6, Batch 10,loss 1.2065573930740356,Validation_accuracy0.2800000011920929
Epoch  6, Batch 11,loss 1.1905494928359985,Validation_accuracy0.2800000011920929
Epoch  6, Batch 12,loss 1.2039562463760376,Validation_accuracy0.2800000011920929
Epoch  6, Batch 13,loss 1.1776500940322876,Validation_accuracy0.2800000011920929
Epoch  6, Batch 14,loss 1.1903531551361084,Validation_accuracy0.2800000011920929
Epoch  6, Batch 15,loss 1.213

Epoch 11, Batch  5,loss 1.2059686183929443,Validation_accuracy0.2800000011920929
Epoch 11, Batch  6,loss 1.1947120428085327,Validation_accuracy0.2800000011920929
Epoch 11, Batch  7,loss 1.1759535074234009,Validation_accuracy0.2800000011920929
Epoch 11, Batch  8,loss 1.2012507915496826,Validation_accuracy0.2800000011920929
Epoch 11, Batch  9,loss 1.160069465637207,Validation_accuracy0.2800000011920929
Epoch 11, Batch 10,loss 1.2062746286392212,Validation_accuracy0.2800000011920929
Epoch 11, Batch 11,loss 1.190309762954712,Validation_accuracy0.2800000011920929
Epoch 11, Batch 12,loss 1.20371413230896,Validation_accuracy0.2800000011920929
Epoch 11, Batch 13,loss 1.177456259727478,Validation_accuracy0.2800000011920929
Epoch 11, Batch 14,loss 1.1900643110275269,Validation_accuracy0.2800000011920929
Epoch 11, Batch 15,loss 1.2128045558929443,Validation_accuracy0.2800000011920929
Epoch 11, Batch 16,loss 1.1928510665893555,Validation_accuracy0.2800000011920929
Epoch 11, Batch 17,loss 1.2046592

Epoch 16, Batch  7,loss 1.1757309436798096,Validation_accuracy0.2800000011920929
Epoch 16, Batch  8,loss 1.2009954452514648,Validation_accuracy0.2800000011920929
Epoch 16, Batch  9,loss 1.159835696220398,Validation_accuracy0.2800000011920929
Epoch 16, Batch 10,loss 1.2059921026229858,Validation_accuracy0.2800000011920929
Epoch 16, Batch 11,loss 1.1900699138641357,Validation_accuracy0.2800000011920929
Epoch 16, Batch 12,loss 1.2034721374511719,Validation_accuracy0.2800000011920929
Epoch 16, Batch 13,loss 1.177262544631958,Validation_accuracy0.2800000011920929
Epoch 16, Batch 14,loss 1.1897755861282349,Validation_accuracy0.2800000011920929
Epoch 16, Batch 15,loss 1.2125235795974731,Validation_accuracy0.2800000011920929
Epoch 16, Batch 16,loss 1.192570686340332,Validation_accuracy0.2800000011920929
Epoch 16, Batch 17,loss 1.204401969909668,Validation_accuracy0.2800000011920929
Epoch 16, Batch 18,loss 1.1941107511520386,Validation_accuracy0.2800000011920929
Epoch 16, Batch 19,loss 1.215941

Epoch 21, Batch  9,loss 1.159602403640747,Validation_accuracy0.2800000011920929
Epoch 21, Batch 10,loss 1.2057098150253296,Validation_accuracy0.2800000011920929
Epoch 21, Batch 11,loss 1.1898304224014282,Validation_accuracy0.2800000011920929
Epoch 21, Batch 12,loss 1.203230619430542,Validation_accuracy0.2800000011920929
Epoch 21, Batch 13,loss 1.1770689487457275,Validation_accuracy0.2800000011920929
Epoch 21, Batch 14,loss 1.189487099647522,Validation_accuracy0.2800000011920929
Epoch 21, Batch 15,loss 1.212242603302002,Validation_accuracy0.2800000011920929
Epoch 21, Batch 16,loss 1.1922905445098877,Validation_accuracy0.2800000011920929
Epoch 21, Batch 17,loss 1.2041451930999756,Validation_accuracy0.2800000011920929
Epoch 21, Batch 18,loss 1.1938592195510864,Validation_accuracy0.2800000011920929
Epoch 21, Batch 19,loss 1.2156585454940796,Validation_accuracy0.2800000011920929
Epoch 21, Batch 20,loss 1.2047548294067383,Validation_accuracy0.2800000011920929
Epoch 22, Batch  1,loss 1.184834

Epoch 26, Batch 11,loss 1.1895911693572998,Validation_accuracy0.2800000011920929
Epoch 26, Batch 12,loss 1.202989101409912,Validation_accuracy0.2800000011920929
Epoch 26, Batch 13,loss 1.1768757104873657,Validation_accuracy0.2800000011920929
Epoch 26, Batch 14,loss 1.1891989707946777,Validation_accuracy0.2800000011920929
Epoch 26, Batch 15,loss 1.211962103843689,Validation_accuracy0.2800000011920929
Epoch 26, Batch 16,loss 1.192010521888733,Validation_accuracy0.2800000011920929
Epoch 26, Batch 17,loss 1.2038884162902832,Validation_accuracy0.2800000011920929
Epoch 26, Batch 18,loss 1.193608045578003,Validation_accuracy0.2800000011920929
Epoch 26, Batch 19,loss 1.2153761386871338,Validation_accuracy0.2800000011920929
Epoch 26, Batch 20,loss 1.2044880390167236,Validation_accuracy0.2800000011920929
Epoch 27, Batch  1,loss 1.1846226453781128,Validation_accuracy0.2800000011920929
Epoch 27, Batch  2,loss 1.1821057796478271,Validation_accuracy0.2800000011920929
Epoch 27, Batch  3,loss 1.202572

Epoch 31, Batch 13,loss 1.1766825914382935,Validation_accuracy0.2800000011920929
Epoch 31, Batch 14,loss 1.188910961151123,Validation_accuracy0.2800000011920929
Epoch 31, Batch 15,loss 1.2116814851760864,Validation_accuracy0.2800000011920929
Epoch 31, Batch 16,loss 1.1917308568954468,Validation_accuracy0.2800000011920929
Epoch 31, Batch 17,loss 1.2036319971084595,Validation_accuracy0.2800000011920929
Epoch 31, Batch 18,loss 1.193356990814209,Validation_accuracy0.2800000011920929
Epoch 31, Batch 19,loss 1.215093970298767,Validation_accuracy0.2800000011920929
Epoch 31, Batch 20,loss 1.2042213678359985,Validation_accuracy0.2800000011920929
Epoch 32, Batch  1,loss 1.184410810470581,Validation_accuracy0.2800000011920929
Epoch 32, Batch  2,loss 1.1818804740905762,Validation_accuracy0.2800000011920929
Epoch 32, Batch  3,loss 1.2022823095321655,Validation_accuracy0.2800000011920929
Epoch 32, Batch  4,loss 1.20733642578125,Validation_accuracy0.2800000011920929
Epoch 32, Batch  5,loss 1.20480489

Epoch 36, Batch 15,loss 1.2114014625549316,Validation_accuracy0.2800000011920929
Epoch 36, Batch 16,loss 1.1914515495300293,Validation_accuracy0.2800000011920929
Epoch 36, Batch 17,loss 1.2033756971359253,Validation_accuracy0.2800000011920929
Epoch 36, Batch 18,loss 1.1931060552597046,Validation_accuracy0.2800000011920929
Epoch 36, Batch 19,loss 1.2148120403289795,Validation_accuracy0.2800000011920929
Epoch 36, Batch 20,loss 1.2039549350738525,Validation_accuracy0.2800000011920929
Epoch 37, Batch  1,loss 1.1841990947723389,Validation_accuracy0.2800000011920929
Epoch 37, Batch  2,loss 1.1816554069519043,Validation_accuracy0.2800000011920929
Epoch 37, Batch  3,loss 1.201992154121399,Validation_accuracy0.2800000011920929
Epoch 37, Batch  4,loss 1.2070733308792114,Validation_accuracy0.2800000011920929
Epoch 37, Batch  5,loss 1.2045283317565918,Validation_accuracy0.2800000011920929
Epoch 37, Batch  6,loss 1.1933790445327759,Validation_accuracy0.2800000011920929
Epoch 37, Batch  7,loss 1.174

Epoch 41, Batch 17,loss 1.2031195163726807,Validation_accuracy0.2800000011920929
Epoch 41, Batch 18,loss 1.1928553581237793,Validation_accuracy0.2800000011920929
Epoch 41, Batch 19,loss 1.214530110359192,Validation_accuracy0.2800000011920929
Epoch 41, Batch 20,loss 1.203688621520996,Validation_accuracy0.2800000011920929
Epoch 42, Batch  1,loss 1.1839873790740967,Validation_accuracy0.2800000011920929
Epoch 42, Batch  2,loss 1.1814303398132324,Validation_accuracy0.2800000011920929
Epoch 42, Batch  3,loss 1.2017024755477905,Validation_accuracy0.2800000011920929
Epoch 42, Batch  4,loss 1.2068103551864624,Validation_accuracy0.2800000011920929
Epoch 42, Batch  5,loss 1.2042518854141235,Validation_accuracy0.2800000011920929
Epoch 42, Batch  6,loss 1.1931232213974,Validation_accuracy0.2800000011920929
Epoch 42, Batch  7,loss 1.1745779514312744,Validation_accuracy0.2800000011920929
Epoch 42, Batch  8,loss 1.1996710300445557,Validation_accuracy0.2800000011920929
Epoch 42, Batch  9,loss 1.1586239

Epoch 46, Batch 19,loss 1.2142484188079834,Validation_accuracy0.2800000011920929
Epoch 46, Batch 20,loss 1.2034225463867188,Validation_accuracy0.2800000011920929
Epoch 47, Batch  1,loss 1.1837756633758545,Validation_accuracy0.2800000011920929
Epoch 47, Batch  2,loss 1.18120539188385,Validation_accuracy0.2800000011920929
Epoch 47, Batch  3,loss 1.2014126777648926,Validation_accuracy0.2800000011920929
Epoch 47, Batch  4,loss 1.206547498703003,Validation_accuracy0.2800000011920929
Epoch 47, Batch  5,loss 1.2039755582809448,Validation_accuracy0.2800000011920929
Epoch 47, Batch  6,loss 1.192867398262024,Validation_accuracy0.2800000011920929
Epoch 47, Batch  7,loss 1.1743565797805786,Validation_accuracy0.2800000011920929
Epoch 47, Batch  8,loss 1.1994168758392334,Validation_accuracy0.2800000011920929
Epoch 47, Batch  9,loss 1.1583914756774902,Validation_accuracy0.2800000011920929
Epoch 47, Batch 10,loss 1.2042449712753296,Validation_accuracy0.2800000011920929
Epoch 47, Batch 11,loss 1.188587

In [26]:
test_acc = sess.run(accuracy,feed_dict={
    x:test_tensors[:],
    y:test_targets[:],
    keep_prob:1.
})
print('Testing accuracy {}'.format(test_acc))

RuntimeError: Attempted to use a closed Session.